In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/keywords.csv


In [2]:
df1=pd.read_csv('../input/the-movies-dataset/credits.csv')
df2=pd.read_csv('../input/the-movies-dataset/movies_metadata.csv')
df4=pd.read_csv('../input/the-movies-dataset/keywords.csv')


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df2 = df2[df2.id!='1997-08-20']
df2 = df2[df2.id!='2012-09-29']
df2 = df2[df2.id!='2014-01-01']

df2['id'] = df2['id'].astype(int)

df2=df2.merge(df1, on='id')

print("Wecome to Movie Buff")
choice=int(input("Choose which sort of recommendation you need:\n1.Similar movie based on a particular movie \n2.Based on movie popularity and/or genre \n"))

if(choice==1):
    tfidf = TfidfVectorizer(stop_words='english')
    df2['overview'] = df2['overview'].fillna('')
    ran = random.randint(25000, 30000)
    df3 = df2.head(ran)

    tfidf_matrix = tfidf.fit_transform(df3['overview'])

    cosine_sim =  linear_kernel(tfidf_matrix, tfidf_matrix, True)

    indices = pd.Series(df3.index, index=df3['title']).drop_duplicates()

    def get_recommendations(title, cosine_sim=cosine_sim):
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movie_indices = [i[0] for i in sim_scores]

        return df3['title'].iloc[movie_indices]


    df2 = df2.merge(df4, on='id')


    features = ['cast', 'crew', 'keywords', 'genres']
    for feature in features:
        df2[feature] = df2[feature].apply(literal_eval)

    def get_director(x):
        for i in x:
            if i['job'] == 'Director':
                return i['name']
        return np.nan

    def get_list(x):
        if isinstance(x, list):
            names = [i['name'] for i in x]
            if len(names) > 3:
                names = names[:3]
            return names

    
        return []


    df2['director'] = df2['crew'].apply(get_director)

    features = ['cast', 'keywords', 'genres']
    for feature in features:
        df2[feature] = df2[feature].apply(get_list)


    def clean_data(x):
        if isinstance(x, list):
            return [str.lower(i.replace(" ", "")) for i in x]
        else:
        
            if isinstance(x, str):
                return str.lower(x.replace(" ", ""))
            else:
                return ''


    features = ['cast', 'keywords', 'director', 'genres']

    for feature in features:
        df2[feature] = df2[feature].apply(clean_data)


    def create_soup(x):
        return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
    df2['soup'] = df2.apply(create_soup, axis=1)

    count = CountVectorizer(stop_words='english')

 
    df5 = df2['soup'].head(20000)

    count_matrix = count.fit_transform(df5)

    cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

    df2 = df2.reset_index()
    indices = pd.Series(df2.index, index=df2['title'])
    
    mv_cnt=int(input("Do you wanna see the list of movies here, if so how many u wanna see: "))
    print(df3['title'].head(mv_cnt))
    mv=input("Enter the movie: ")
    print(get_recommendations(mv, cosine_sim2))

if(choice==2):

    C= df2['vote_average'].mean()
    m= df2['vote_count'].quantile(0.9)

    q_movies = df2.copy().loc[df2['vote_count'] >= m]

    def weighted_rating(x, m=m, C=C):
        v = x['vote_count']
        R = x['vote_average']

        return (v/(v+m) * R) + (m/(m+v) * C)

    q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

    q_movies = q_movies.sort_values('score', ascending=False)


    n=int(input("List of how much top movies do you want: "))
    print(q_movies[['title', 'vote_count', 'vote_average', 'score']].head(n))


Wecome to Movie Buff
Choose which sort of recommendation you need:
1.Similar movie based on a particular movie 
2.Based on movie popularity and/or genre 
1
Do you wanna see the list of movies here, if so how many u wanna see: 17
0                       Toy Story
1                         Jumanji
2                Grumpier Old Men
3               Waiting to Exhale
4     Father of the Bride Part II
5                            Heat
6                         Sabrina
7                    Tom and Huck
8                    Sudden Death
9                       GoldenEye
10         The American President
11    Dracula: Dead and Loving It
12                          Balto
13                          Nixon
14               Cutthroat Island
15                         Casino
16          Sense and Sensibility
Name: title, dtype: object
Enter the movie: Assassins
2311                   Back to School
3106     Batman: Mask of the Phantasm
4532                 Bread and Tulips
8787                Trilo